<a href="https://colab.research.google.com/github/kimyeulin/kimyeulin.github.io/blob/master/nextSentence_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.42.178.242:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 12941694308483178525),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 4600599932543810177),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 922326608497122751),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 4689122703447443482),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 17950063578963247113),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 14909318735090019586),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 1148472808558510234),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 8618577336530150283),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 5873239878390

In [13]:
pip install pytorch-pretrained-bert

In [0]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM,BertForNextSentencePrediction

In [0]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [0]:


# Tokenized input
text = "[CLS] When photography came along in the nineteenth century, painting was put in crisis [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = tokenizer.tokenize(text)

# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

# Load pre-trained model (weights)
model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')
model.eval()

# Predict is Next Sentence ?
predictions = model(tokens_tensor, segments_tensors )

In [0]:
# Tokenized input
text = "[CLS] When photography came along in the nineteenth century, painting was put in crisis [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = tokenizer.tokenize(text)
tokenized_text.split('[SEP]', maxsplit=2)

In [0]:
predictions

tensor([[-3.9709,  6.9328]], grad_fn=<AddmmBackward>)

In [0]:
def tokenizing(text_A, text_B):
  text_A.strip()
  text_A = '[CLS] ' + text_A + ' [SEP]'
  text_B.strip()
  text_B = text_B + ' [SEP]'
  
  tokenized_text_A = tokenizer.tokenize(text_A)
  tokenized_text_B = tokenizer.tokenize(text_B)
  
  segments_ids = []
  for i in range(len(tokenized_text_A)):
    segments_ids.append(0)
  for i in range(len(tokenized_text_B)):
    segments_ids.append(1)

  return tokenized_text_A + tokenized_text_B, segments_ids

def toTensors(text_A, text_B):
  tokenized_text, segments_ids = tokenizing(text_A, text_B)
  print(tokenized_text)
  print(segments_ids)
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])
  
  return tokens_tensor, segments_tensors 

In [40]:
# Load pre-trained model (weights)
model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')
model.eval()

tokens_tensor, segments_tensors = toTensors('But by and large, the photograph was a challenge to painting and was one cause of painting’s moving away from direct representation and reproduction to the abstract painting of the twentieth century.', 
                                            'Therefore, the painters of that century put more focus on expressing nature, people, and cities as they were in reality.')

# Predict
predictions = model(tokens_tensor, segments_tensors)

['[CLS]', 'but', 'by', 'and', 'large', ',', 'the', 'photograph', 'was', 'a', 'challenge', 'to', 'painting', 'and', 'was', 'one', 'cause', 'of', 'painting', '’', 's', 'moving', 'away', 'from', 'direct', 'representation', 'and', 'reproduction', 'to', 'the', 'abstract', 'painting', 'of', 'the', 'twentieth', 'century', '.', '[SEP]', 'therefore', ',', 'the', 'painters', 'of', 'that', 'century', 'put', 'more', 'focus', 'on', 'expressing', 'nature', ',', 'people', ',', 'and', 'cities', 'as', 'they', 'were', 'in', 'reality', '.', '[SEP]']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [41]:
softmax = torch.nn.Softmax(dim=1)
softmax(predictions)

tensor([[1.0000e+00, 4.4705e-06]], grad_fn=<SoftmaxBackward>)

In [30]:
predictions

tensor([[-4.0971,  7.1509]], grad_fn=<AddmmBackward>)